# 📝 Exercise M3.02

The goal is to find the best set of hyperparameters which maximize the
statistical performance on a training set.

Here again with limit the size of the training set to make computation
run faster. Feel free to increase the `train_size` value if your computer
is powerful enough.

In [1]:
import numpy as np
import pandas as pd

adult_census = pd.read_csv("../datasets/adult-census.csv")

target_name = "class"
target = adult_census[target_name]
data = adult_census.drop(columns=[target_name, "education-num"])

from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(
    data, target, random_state=42)

Create your machine learning pipeline

You should:
* preprocess the categorical columns using a `OneHotEncoder` and use a
  `StandardScaler` to normalize the numerical data.
* use a `LogisticRegression` as a predictive model.

Start by defining the columns and the preprocessing pipelines to be applied
on each columns.

In [24]:
from sklearn.compose import make_column_selector as selector

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(data)
categorical_columns = categorical_columns_selector(data)

categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

Subsequently, create a `ColumnTransformer` to redirect the specific columns
a preprocessing pipeline.

In [25]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    [
        ('one-hot-encoder', categorical_preprocessor, categorical_columns),
        ('standard-scaler', numerical_preprocessor, numerical_columns)
    ]
)

Finally, concatenate the preprocessing pipeline with a logistic regression.

In [26]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# make_pipeline generates names for steps automatically comp Pipeline

pipeline = make_pipeline(
    preprocessor,
    LogisticRegression()
)

Use a `RandomizedSearchCV` to find the best set of hyperparameters by tuning
the following parameters of the `model`:

- the parameter `C` of the `LogisticRegression` with values ranging from
  0.001 to 10. You can use a log-uniform distribution
  (i.e. `scipy.stats.loguniform`);
- the parameter `with_mean` of the `StandardScaler` with possible values
  `True` or `False`;
- the parameter `with_std` of the `StandardScaler` with possible values
  `True` or `False`.

Once the computation has completed, print the best combination of parameters
stored in the `best_params_` attribute.

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('one-hot-encoder', OneHotEncoder(),
                                    ['workclass', 'education', 'marital-status',
                                     'occupation', 'relationship', 'race', 'sex',
                                     'native-country']),
                                   ('standard-scaler', StandardScaler(),
                                    ['age', 'capital-gain', 'capital-loss',
                                     'hours-per-week'])])),
  ('logisticregression', LogisticRegression(max_iter=500))],
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('one-hot-encoder', OneHotEncoder(),
                                  ['workclass', 'education', 'marital-status',
                                   'occupation', 'relationship', 'race', 'sex',
                                   'native-country']),
                                 ('standard-scaler', StandardSca

In [27]:
from scipy.stats import loguniform

from sklearn.model_selection import RandomizedSearchCV

param_distributions = {
    'logisticregression__C': loguniform(0.001, 10),
    'columntransformer__standard-scaler__with_mean': [True, False],
    'columntransformer__standard-scaler__with_std': [True, False],
}

model_random_search = RandomizedSearchCV(
    pipeline,
    param_distributions=param_distributions,
    n_iter=20,
    verbose=1,
)

model_random_search.fit(data_train, target_train)

EACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\WIECKOWS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\WIECKOWS\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\linear_model\_logistic.py:7

RandomizedSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('one-hot-encoder',
                                                                               OneHotEncoder(handle_unknown='ignore'),
                                                                               ['workclass',
                                                                                'education',
                                                                                'marital-status',
                                                                                'occupation',
                                                                                'relationship',
                                                                                'race',
                                                                                'sex',
                                                                   

In [28]:
accuracy = model_random_search.score(data_test, target_test)

print(f"The test accuracy score of the best model is "
      f"{accuracy:.2f}")

print(
    "The best parameters are:",
    model_random_search.best_params_
)

The test accuracy score of the best model is 0.86
The best parameters are: {'columntransformer__standard-scaler__with_mean': True, 'columntransformer__standard-scaler__with_std': True, 'logisticregression__C': 0.8233319171810176}
